# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys


    

In [150]:
data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [151]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [152]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоватьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [ ]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),  # На вход item-user matrix
          show_progress=True)

-----

# Домашнее задание

1. Перенесите метрики, которые мы реализовывали в модуль metrics.py
2. Допишите функцию prefilter_items в модуль utils.py
3. Допишите код для класса MainRecommender в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [145]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:

    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    weighting: string
        Тип взвешивания, один из вариантов None, "bm25", "tfidf"
    fake_id: int
        идентификатор, которым заменялись редкие объекты, можно передать None?если такого объекта нет
    """

    def __init__(self, data, weighting=None, fake_id = 999999):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != fake_id]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != fake_id]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.fake_id = fake_id
        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)
        self.user_item_matrix = csr_matrix(self.user_item_matrix).tocsr()
        self.user_item_matrix_for_pred = self.user_item_matrix
        #Можно добавить дополнительное взвешивание
        if weighting == "bm25":
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T.tocsr()   
        elif weighting == "tfidf":
            self.user_item_matrix = tfidf_weight(self.user_item_matrix.T).T.tocsr()

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):        
        """Готовит user-item матрицу"""
#         popularity = data.groupby('item_id')['quantity'].sum().reset_index()
#         popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

#         top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
        # Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
#         data.loc[~data['item_id'].isin(top_5000), 'item_id'] = 999999
        
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        
        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(user_item_matrix)

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.01, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(user_item_matrix)

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1
            
            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[0][1]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]
    
    def _get_similar_user(self, user_id, N):
        """Находит юзеров, похожих на user_id"""
        recs = self.model.similar_users(self.userid_to_id[user_id], N+1)  # Юзер похож на себя -> рекомендуем N+1 юзера
        top_rec = recs[0][1:]  # И берем со второго (не юзер из аргумента метода)
#         for i in range(len(top_rec)):
#             top_rec[i] = self.id_to_userid[top_rec]
        return [(lambda x:self.id_to_userid[x])(x) for x in top_rec]
#             self.id_to_userid[top_rec[:]]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            top_popular = [rec for rec in self.overall_top_purchases[:N] if rec not in recommendations and rec!=self.fake_id]
            recommendations.extend(top_popular)
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""
        self._update_dict(user_id=user)
#         filter_items = [] if self.fake_id is None else [self.itemid_to_id[self.fake_id]]
        res = model.recommend(userid=self.userid_to_id[user],
            user_items=self.user_item_matrix_for_pred[self.userid_to_id[user]],
            N=N,
            filter_already_liked_items=False,
            filter_items=[self.itemid_to_id[999999]],
            recalculate_user=True)
        mask = res[1].argsort()[::-1]
        res = [self.id_to_itemid[rec] for rec in res[0][mask]]
        res = self._extend_with_top_popular(res, N=N)
            
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""
        if user not in self.userid_to_id:
            return self._extend_with_top_popular([], N=N)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""
        if user not in self.userid_to_id:
            return self._extend_with_top_popular([], N=N)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        #your code
        res = []
        top_n = self.top_purchases[self.top_purchases['user_id']==user][:N]['item_id'].tolist()
        if top_n is not None:
            for item in top_n:
                similar_item = self._get_similar_item(item)
                if similar_item != self.fake_id:                    
                    res.append(similar_item) 
        if len(res) < N:
            """Если не получилось найти нужное количество похожих товаров, то добавим в список самые популярные товары"""
            res = self._extend_with_top_popular(res, N=N)       

#         assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        #your code
        res = []
#         similar_users_list = []
        if user in self.userid_to_id:
            similar_users_list = self._get_similar_user(user, N) #.tolist()
        for u in similar_users_list:
            top_user = self.top_purchases[self.top_purchases['user_id']==u]['item_id'].tolist()
            if len(top_user) != 0:
                res.append(top_user[0])
        if len(res) < N:
            """Если не получилось найти нужное количество товаров, то добавим в список самые популярные товары"""
            res = self._extend_with_top_popular(res, N=N)
        return res

----

Проверка, что все работает

In [153]:
module_path = os.path.abspath(os.path.join(os.pardir))

In [168]:
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from best_rec_lib.metrics import precision_at_k, recall_at_k
from best_rec_lib.utils import prefilter_items
from best_rec_lib.recommenders import MainRecommender

Попробуйте сделать предсказание с помощью нового класса

In [169]:
data_pr = prefilter_items(data_train)

In [170]:
recomends = MainRecommender(data_pr)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [171]:
%%time
    
result['main_rec_own'] = result['user_id'].apply(lambda x: recomends.get_own_recommendations(x))
print('precision_at_5: ', result.apply(lambda row: precision_at_k(row['main_rec_own'], row['actual']), axis=1).mean())
print('recall_at_5: ', result.apply(lambda row: recall_at_k(row['main_rec_own'], row['actual']), axis=1).mean())

precision_at_5:  0.20558276199804118
recall_at_5:  0.030878660673682088
Wall time: 387 ms


In [172]:
%%time
    
result['als_rec'] = result['user_id'].apply(lambda x: recomends.get_als_recommendations(x))
print('precision_at_5: ', result.apply(lambda row: precision_at_k(row['als_rec'], row['actual']), axis=1).mean())
print('recall_at_5: ',result.apply(lambda row: recall_at_k(row['als_rec'], row['actual']), axis=1).mean())

precision_at_5:  0.1851126346718903
recall_at_5:  0.02898166660680219
Wall time: 806 ms


In [173]:
%%time
    
result['similar_item'] = result['user_id'].apply(lambda x: recomends.get_similar_items_recommendation(x))
print('precision_at_5: ', result.apply(lambda row: precision_at_k(row['similar_item'], row['actual']), axis=1).mean())
print('recall_at_5: ',result.apply(lambda row: recall_at_k(row['similar_item'], row['actual']), axis=1).mean())

precision_at_5:  0.1682664054848188
recall_at_5:  0.019226075862305708
Wall time: 2.78 s


In [174]:
%%time
    
result['similar_user'] = result['user_id'].apply(lambda x: recomends.get_similar_users_recommendation(x))
print('precision_at_5: ', result.apply(lambda row: precision_at_k(row['similar_user'], row['actual']), axis=1).mean())
print('recall_at_5: ',result.apply(lambda row: recall_at_k(row['similar_user'], row['actual']), axis=1).mean())

precision_at_5:  0.1979431929480901
recall_at_5:  0.020171123799730628
Wall time: 8.98 s


In [175]:
result.head()

,user_id,actual,main_rec_own,als_rec,similar_item,similar_user
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 6534178, 1029743, 995242, 1106523]","[862349, 1070820, 981760, 995242, 1082185]","[990335, 1029743, 826597, 15926844, 1082185]","[6534178, 1082185, 862349, 1082185, 962229]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 6534178, 1029743, 995242, 1106523]","[8090521, 951590, 962229, 883404, 5569230]","[1076875, 1092026, 910032, 1138953, 1133018]","[6534178, 951590, 8090521, 1082185, 986394]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 6534178, 1029743, 995242, 1106523]","[1024306, 854852, 834484, 878996, 860776]","[825541, 904360, 845208, 948650, 1082185]","[1082185, 1082185, 1082185, 1082185, 1028816]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 6534178, 1029743, 995242, 1106523]","[1082185, 6534178, 1029743, 995242, 849843]","[1019247, 1015247, 1094955, 9488055, 1082185]","[1126899, 1129822, 1070820, 923746, 6534178]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 6534178, 1029743, 995242, 1106523]","[826249, 1098066, 883404, 840361, 1068719]","[1021324, 1106523, 1110843, 1053690, 1082185]","[1070820, 1082185, 1070820, 1029743, 923746]"
